<a href="https://colab.research.google.com/github/Rishabh690/Banking-_chatbot-/blob/main/Chatbot_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# !pip install Transformers


In [12]:
df = pd.read_csv("/content/BankFAQs.csv")

In [13]:
df.head()

,Question,Answer,Class
0,Do I need to enter ‘#’ after keying in my Card...,Please listen to the recorded message and foll...,security
1,What details are required when I want to perfo...,"To perform a secure IVR transaction, you will ...",security
2,How should I get the IVR Password if I hold a...,An IVR password can be requested only from the...,security
3,How do I register my Mobile number for IVR Pas...,Please call our Customer Service Centre and en...,security
4,How can I obtain an IVR Password,By Sending SMS request: Send an SMS 'PWD<space...,security


In [14]:
df.count()

Question    1764
Answer      1764
Class       1764
dtype: int64

In [15]:
df["Class"].unique()

array(['security', 'loans', 'accounts', 'insurance', 'investments',
       'fundstransfer', 'cards'], dtype=object)

In [16]:
df=df.drop(["Class"],axis=1)

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1764 entries, 0 to 1763
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Question  1764 non-null   object
 1   Answer    1764 non-null   object
dtypes: object(2)
memory usage: 27.7+ KB


In [18]:
df.head()

,Question,Answer
0,Do I need to enter ‘#’ after keying in my Card...,Please listen to the recorded message and foll...
1,What details are required when I want to perfo...,"To perform a secure IVR transaction, you will ..."
2,How should I get the IVR Password if I hold a...,An IVR password can be requested only from the...
3,How do I register my Mobile number for IVR Pas...,Please call our Customer Service Centre and en...
4,How can I obtain an IVR Password,By Sending SMS request: Send an SMS 'PWD<space...


In [19]:
from nltk.tokenize import word_tokenize
df["Question"]=df["Question"].str.lower()


In [20]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [21]:
import re
text=df["Question"][0]
from nltk.tokenize import RegexpTokenizer
text = re.sub(r'[^\w\s]', '', text)
words = word_tokenize(text)


In [22]:
words

['do',
 'i',
 'need',
 'to',
 'enter',
 'after',
 'keying',
 'in',
 'my',
 'card',
 'number',
 'card',
 'expiry',
 'date',
 'cvv',
 'number']

In [23]:
!pip install torch
import torch
!pip install Transformers==3.2.0
from transformers import BertTokenizer, BertModel


NotImplementedError: ignored

# Bert to generate embeddings

In [24]:
from transformers import BertTokenizer, BertModel
import torch

# Load the pre-trained BERT model and tokenizer
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

# Define a function to get BERT embeddings for questions
def get_question_embeddings(questions):
    inputs = tokenizer(questions, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings

question_embeddings = get_question_embeddings(df["Question"].tolist())

# question_embeddings will be a torch tensor containing embeddings for all questions in the dataset
print(question_embeddings.shape)  # Output: torch.Size([3, max_tokens, 768])


torch.Size([1764, 768])


# T5 for data augmentation

In [6]:
!pip install Sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.3 MB/s eta 0:00:00


In [8]:
!pip install transformers
from transformers import T5ForConditionalGeneration,T5Tokenizer
from sklearn.metrics.pairwise import cosine_similarity
import torch

model = T5ForConditionalGeneration.from_pretrained('ramsrigouthamg/t5_paraphraser')
tokenizer = T5Tokenizer.from_pretrained('t5-base')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print ("device ",device)
model = model.to(device)
sentence = "How can I obtain an IVR Password?	"
text =  "paraphrase: " + sentence + " </s>"
max_len = 256
encoding = tokenizer.encode_plus(text,pad_to_max_length=True, return_tensors="pt")
input_ids, attention_masks = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)


# set top_k = 50 and set top_p = 0.95 and num_return_sequences = 3
beam_outputs = model.generate(
    input_ids=input_ids, attention_mask=attention_masks,
    do_sample=True,
    max_length=256,
    top_k=120,
    top_p=0.98,
    early_stopping=True,
    num_return_sequences=10
)


print ("\nOriginal Question ::")
print (sentence)
print ("\n")
print ("Paraphrased Questions :: ")
final_outputs =[]
for beam_output in beam_outputs:
    sent = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
    if sent.lower() != sentence.lower() and sent not in final_outputs:
        final_outputs.append(sent)

for i, final_output in enumerate(final_outputs):
    print("{}: {}".format(i, final_output))


/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:199: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


device  cuda


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2393: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:262: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(



Original Question ::
How can I obtain an IVR Password?	


Paraphrased Questions :: 
0: How do I find the IVR password?
1: What are the best ways to obtain an IVR password?
2: How can I get an IVR password?
3: How can I use an IVR password?
4: How can I obtain my IVR password?
5: Can I get a Password for IVR?
6: How do I obtain an IVR password?
7: How do I obtain a IVR phone number?
8: How do I re-enable an IVR password?
9: How can I get a IVR password?


In [29]:
import numpy as np
from collections import Counter

def calculate_cosine_similarity(embedding1, embedding2):
    dot_product = np.dot(embedding1, embedding2.T)  # Transpose embedding2 to make the dot product work
    norm_embedding1 = np.linalg.norm(embedding1)
    norm_embedding2 = np.linalg.norm(embedding2)
    return dot_product / (norm_embedding1 * norm_embedding2)

def get_most_similar_question(user_embedding, question_embeddings):
    similarity_scores = []
    for question_embedding in question_embeddings:
        similarity_score = calculate_cosine_similarity(user_embedding, question_embedding)
        similarity_scores.append(similarity_score)

    most_similar_index = np.argmax(similarity_scores)
    return most_similar_index

# Sample user input (replace this with actual user input)
user_embedding = get_question_embeddings(final_outputs)

# Sample question embeddings (replace this with actual question embeddings)

# Assuming question_embeddings is a list of embeddings with shape (1, 768)
# Flatten each embedding in the list to make them 1-dimensional
question_embeddings = [embedding.flatten() for embedding in question_embeddings]

# Find the most similar question for each user embedding
most_similar_indices = [get_most_similar_question(embedding, question_embeddings) for embedding in user_embedding]

# Count the occurrences of each question index in the most_similar_indices array
question_count = Counter(most_similar_indices)

# Find the index of the question that appears the most frequently
most_frequent_question_index = question_count.most_common(1)[0][0]

# Retrieve the corresponding answer for the most frequent question
response = df["Answer"].iloc[most_frequent_question_index]
print("Chatbot:", response)


<ipython-input-29-1101136c7b17>:5: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3571.)
  dot_product = np.dot(embedding1, embedding2.T)  # Transpose embedding2 to make the dot product work


Chatbot: By Sending SMS request: Send an SMS 'PWD<space>1234' to 9717465555 or to 5676712 from your registered (with Bank) mobile number.(Note: 1234 are the last 4 digits of your HDFC Bank Credit Card number). You will receive an SMS with the IVR password on the same number. From HDFC Bank Website: If you have registered your card for NetSafe/ Verified by Visa/ MasterCard SecureCode, you can also login in to your NetSafe/ Verified by Visa/ MasterCard SecureCode account and use the Generate IVR Password option available on the left menu. The IVR password will be sent to your registered mobile number and email ID. These are the most convenient and recommended options. To ensure convenience, make a note of the IVR password and keep it handy while performing the transaction. Note: Kindly ensure that your latest mobile number and email ID is updated with us. Premium SMS charges as per your mobile service provider will apply for an SMS sent to 5676712 View more
